In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, median_absolute_error
import matplotlib.pyplot as plt

# Function to load and process flight data
def load_and_process_flight_data(file_paths):
    flights = []
    for file_path in file_paths:
        df = pd.read_csv(file_path, delimiter='\t')
        if 'Time(milli)' in df.columns:
            df = df.drop(columns=['Time(milli)'])
        flights.append(df)
    return flights

# Function to create subsequences from flight data for training
def create_subsequences(flight_data, seq_length):
    state_subsequences = []
    action_subsequences = []
    for df in flight_data:
        states = df[['LAT(deg)', 'LON(deg)', 'ALT(m)', 'Phi(deg)', 'Theta(deg)', 'Psi(deg)',
                     'Vx(m/s)', 'Vy(m/s)', 'Vz(m/s)', 'P(deg/s)', 'Q(deg/s)', 'R(deg/s)',
                     'Nx(m/s2)', 'Ny(m/s2)', 'Nz(m/s2)', 'Radial(deg)', 'Distance(m)', 'DeltaAlt:Anv-Tgt(m)']].values
        actions = df[['JX', 'JY', 'Throttle']].values
        for i in range(len(states) - seq_length + 1):
            state_subsequences.append(states[i:i+seq_length])
            action_subsequences.append(actions[i:i+seq_length])
    return np.array(state_subsequences), np.array(action_subsequences)

# Function to create complete sequences from flight data for prediction
def create_complete_sequences(flight_data):
    state_sequences = []
    action_sequences = []
    for df in flight_data:
        states = df[['LAT(deg)', 'LON(deg)', 'ALT(m)', 'Phi(deg)', 'Theta(deg)', 'Psi(deg)',
                     'Vx(m/s)', 'Vy(m/s)', 'Vz(m/s)', 'P(deg/s)', 'Q(deg/s)', 'R(deg/s)',
                     'Nx(m/s2)', 'Ny(m/s2)', 'Nz(m/s2)', 'Radial(deg)', 'Distance(m)', 'DeltaAlt:Anv-Tgt(m)']].values
        actions = df[['JX', 'JY', 'Throttle']].values
        state_sequences.append(states)
        action_sequences.append(actions)
    return np.array(state_sequences), np.array(action_sequences)

# Function to plot actual vs predicted actions with mean and standard deviation
def plot_trajectory_with_mean_std(actual_actions_list, predicted_actions_list, plots_dir):
    actual_actions = np.array(actual_actions_list)
    predicted_actions = np.array(predicted_actions_list)

    # Adjust time steps to handle different lengths
    max_time_steps = max(actual_actions.shape[1], predicted_actions.shape[1])
    time_steps = np.arange(max_time_steps)

    actual_mean = np.mean(actual_actions, axis=0)
    actual_std = np.std(actual_actions, axis=0)
    predicted_mean = np.mean(predicted_actions, axis=0)

    plt.figure(figsize=(15, 5))

    # Plot JX
    plt.subplot(1, 3, 1)
    plt.plot(time_steps[:actual_mean.shape[0]], actual_mean[:, 0], label='Actual JX', color='blue')
    plt.fill_between(time_steps[:actual_mean.shape[0]], actual_mean[:, 0] - actual_std[:, 0], actual_mean[:, 0] + actual_std[:, 0], color='blue', alpha=0.2)
    plt.plot(time_steps[:predicted_mean.shape[0]], predicted_mean[:, 0], label='Predicted JX', linestyle='--', color='red')
    plt.xlabel('Time Step')
    plt.ylabel('JX')
    plt.legend()

    # Plot JY
    plt.subplot(1, 3, 2)
    plt.plot(time_steps[:actual_mean.shape[0]], actual_mean[:, 1], label='Actual JY', color='blue')
    plt.fill_between(time_steps[:actual_mean.shape[0]], actual_mean[:, 1] - actual_std[:, 1], actual_mean[:, 1] + actual_std[:, 1], color='blue', alpha=0.2)
    plt.plot(time_steps[:predicted_mean.shape[0]], predicted_mean[:, 1], label='Predicted JY', linestyle='--', color='red')
    plt.xlabel('Time Step')
    plt.ylabel('JY')
    plt.legend()

    # Plot Throttle
    plt.subplot(1, 3, 3)
    plt.plot(time_steps[:actual_mean.shape[0]], actual_mean[:, 2], label='Actual Throttle', color='blue')
    plt.fill_between(time_steps[:actual_mean.shape[0]], actual_mean[:, 2] - actual_std[:, 2], actual_mean[:, 2] + actual_std[:, 2], color='blue', alpha=0.2)
    plt.plot(time_steps[:predicted_mean.shape[0]], predicted_mean[:, 2], label='Predicted Throttle', linestyle='--', color='red')
    plt.xlabel('Time Step')
    plt.ylabel('Throttle')
    plt.legend()

    plt.suptitle('Trajectory Comparison: Actual vs Predicted with Mean and Standard Deviation')
    plt.savefig(os.path.join(plots_dir, 'trajectory_comparison_mean_std.eps'), format='eps')
    plt.savefig(os.path.join(plots_dir, 'trajectory_comparison_mean_std.png'), format='png', dpi=500)
    plt.close()

# Directory containing the adjusted flight data files
adjusted_data_directory = './adjusted_flights/'
adjusted_file_pattern = os.path.join(adjusted_data_directory, 'SimuladorDeVoo_*.txt')
adjusted_files = glob.glob(adjusted_file_pattern)

# Load and process the adjusted flight data
flight_data = load_and_process_flight_data(adjusted_files)

# Verify that data is loaded correctly
print(f'Loaded {len(flight_data)} flight data samples.')
print(f'Example data shape: {flight_data[0].shape if len(flight_data) > 0 else "No data"}')

# Define sequence length for training
sequence_length = 2

# Create subsequences from flight data for training
state_subsequences, action_subsequences = create_subsequences(flight_data, sequence_length)

# Create complete sequences from flight data for prediction
state_sequences, action_sequences = create_complete_sequences(flight_data)

# Normalize the data
state_mean = np.mean(state_subsequences, axis=(0, 1))
state_std = np.std(state_subsequences, axis=(0, 1))
state_subsequences = (state_subsequences - state_mean) / state_std

# Normalize complete sequences using the same mean and std
state_sequences = (state_sequences - state_mean) / state_std

# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_no = 1
metrics_list = []
best_rmse = float('inf')
best_r2 = float('-inf')
best_fold = 0

# Directories to save metrics, models, and plots
metrics_dir = './metrics/'
plots_dir = './plots/'
models_dir = './models/'
os.makedirs(metrics_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

for train_index, test_index in kf.split(state_subsequences):
    states_train, states_test = state_subsequences[train_index], state_subsequences[test_index]
    actions_train, actions_test = action_subsequences[train_index], action_subsequences[test_index]

    # Split training data into training and validation sets
    states_train, states_val, actions_train, actions_val = train_test_split(states_train, actions_train, test_size=0.2, random_state=42)
    
    # Define the model with complexity
    model = Sequential()
    model.add(LSTM(512, input_shape=(sequence_length, states_train.shape[2]), return_sequences=True))
    model.add(TimeDistributed(Dense(256, activation='relu')))    
    model.add(TimeDistributed(Dense(128, activation='relu')))
    model.add(TimeDistributed(Dense(actions_train.shape[2])))

    # Compile the model with a lower learning rate
    optimizer = Adam(learning_rate=0.00001)
    model.compile(optimizer=optimizer, loss='mse')

    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Train the model with validation and early stopping
    history = model.fit(states_train, actions_train, epochs=100_000_000, batch_size=32, validation_data=(states_val, actions_val), callbacks=[early_stopping], verbose=0)

    # Evaluate the model
    loss = model.evaluate(states_test, actions_test)
    print(f'Fold {fold_no} - Test Loss: {loss}')

    # Predict actions on the test set
    predicted_actions = model.predict(states_test, verbose=0)

    # Reshape for metrics calculations
    actions_test_reshaped = actions_test.reshape(-1, actions_test.shape[2])
    predicted_actions_reshaped = predicted_actions.reshape(-1, predicted_actions.shape[2])

    # Calculate MSE, RMSE, MAE, R², MAPE, and MedAE
    mse = mean_squared_error(actions_test_reshaped, predicted_actions_reshaped)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actions_test_reshaped, predicted_actions_reshaped)
    r2 = r2_score(actions_test_reshaped, predicted_actions_reshaped)
    mape = mean_absolute_percentage_error(actions_test_reshaped, predicted_actions_reshaped)
    medae = median_absolute_error(actions_test_reshaped, predicted_actions_reshaped)

    # Append metrics to list
    metrics_list.append({
        'fold': fold_no,
        'loss': loss,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mape': mape,
        'medae': medae
    })

    print(f'Fold {fold_no} - Test MSE: {mse}')
    print(f'Fold {fold_no} - Test RMSE: {rmse}')
    print(f'Fold {fold_no} - Test MAE: {mae}')
    print(f'Fold {fold_no} - Test R²: {r2}')
    print(f'Fold {fold_no} - Test MAPE: {mape}')
    print(f'Fold {fold_no} - Test MedAE: {medae}')

    # Plot training & validation loss values
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Model Loss - Fold {fold_no}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.savefig(os.path.join(plots_dir, f'loss_fold_{fold_no}.png'))
    plt.close()

    # Save the model
    model.save(os.path.join(models_dir, f'model_fold_{fold_no}.keras'))

    # Update best model if current fold's RMSE and R² are the best
    if rmse < best_rmse or (rmse == best_rmse and r2 > best_r2):
        best_rmse = rmse
        best_r2 = r2
        best_fold = fold_no

    fold_no += 1

# Convert metrics list to DataFrame and save as CSV
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(os.path.join(metrics_dir, 'cross_validation_metrics.csv'), index=False)

# Load the best model
best_model_path = os.path.join(models_dir, f'model_fold_{best_fold}.keras')
best_model = load_model(best_model_path)

# Calculate overall metrics and save as CSV
overall_metrics = {
    'mean_mse': [metrics_df['mse'].mean()],
    'std_mse': [metrics_df['mse'].std()],
    'mean_rmse': [metrics_df['rmse'].mean()],
    'std_rmse': [metrics_df['rmse'].std()],
    'mean_mae': [metrics_df['mae'].mean()],
    'std_mae': [metrics_df['mae'].std()],
    'mean_r2': [metrics_df['r2'].mean()],
    'std_r2': [metrics_df['r2'].std()],
    'mean_mape': [metrics_df['mape'].mean()],
    'std_mape': [metrics_df['mape'].std()],
    'mean_medae': [metrics_df['medae'].mean()],
    'std_medae': [metrics_df['medae'].std()],
    'best_fold': [best_fold],
    'best_rmse': [best_rmse],
    'best_r2': [best_r2]
}
overall_metrics_df = pd.DataFrame(overall_metrics)
overall_metrics_df.to_csv(os.path.join(metrics_dir, 'overall_metrics.csv'), index=False)

print(f'Mean MSE: {metrics_df["mse"].mean()}')
print(f'STD MSE: {metrics_df["mse"].std()}')
print(f'Mean RMSE: {metrics_df["rmse"].mean()}')
print(f'STD RMSE: {metrics_df["rmse"].std()}')
print(f'Mean MAE: {metrics_df["mae"].mean()}')
print(f'STD MAE: {metrics_df["mae"].std()}')
print(f'Mean R²: {metrics_df["r2"].mean()}')
print(f'STD R²: {metrics_df["r2"].std()}')
print(f'Mean MAPE: {metrics_df["mape"].mean()}')
print(f'STD MAPE: {metrics_df["mape"].std()}')
print(f'Mean MedAE: {metrics_df["medae"].mean()}')
print(f'STD MedAE: {metrics_df["medae"].std()}')
print(f'Best Fold: {best_fold}')
print(f'Best RMSE: {best_rmse}')
print(f'Best R²: {best_r2}')

# Collect actual and predicted actions
actual_actions_list = []
predicted_actions_list = []

# Function to predict full flight trajectories using a sliding window approach
def predict_full_trajectory(model, state_sequence, sequence_length):
    predicted_actions = []
    num_steps = len(state_sequence)

    for i in range(num_steps - sequence_length + 1):
        window = state_sequence[i:i + sequence_length].reshape(1, sequence_length, state_sequence.shape[1])
        predicted_action = model.predict(window, verbose=0)
        predicted_actions.append(predicted_action[:, -1, :])  # Take the prediction for the last time step

    # Ensure that predicted actions match the original sequence length
    while len(predicted_actions) < num_steps:
        predicted_actions.append(predicted_actions[-1])

    return np.vstack(predicted_actions)

# Predict full flight trajectories using the best model
for i, state_sequence in enumerate(state_sequences):
    predicted_actions_full = predict_full_trajectory(best_model, state_sequence, sequence_length)
    actual_actions_list.append(action_sequences[i])
    predicted_actions_list.append(predicted_actions_full)
    print(f'Full flight trajectory {i} predicted and collected.')

# Plot the mean and standard deviation for the collected trajectories
plot_trajectory_with_mean_std(actual_actions_list, predicted_actions_list, plots_dir)
print('Trajectory comparison plot with mean and standard deviation saved.')

# Explanation of predicted values generation
# The function `predict_full_trajectory` uses a sliding window approach to generate predictions for the entire flight trajectory.
# 1. The input sequence is divided into windows of `sequence_length`.
# 2. Each window is reshaped to match the input shape expected by the model.
# 3. The model predicts the action for the last time step of each window.
# 4. These predicted actions are collected to form the complete predicted trajectory.
# 5. If there are remaining time steps that were not covered by the windows, the last predicted action is repeated to match the original sequence length.

Loaded 30 flight data samples.
Example data shape: (39, 21)
8/8 [==============================] - 0s 3ms/step - loss: 3.0444
Fold 1 - Test Loss: 3.044358968734741
Fold 1 - Test MSE: 3.0443590244780867
Fold 1 - Test RMSE: 1.7448091656333327
Fold 1 - Test MAE: 0.680874758214847
Fold 1 - Test R²: 0.7677202618660398
Fold 1 - Test MAPE: 2354956110082.2285
Fold 1 - Test MedAE: 0.4439247182259874
